In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_columns", None)

In [ ]:
def unzip(path_to_zip_file, directory_to_extract_to):
    """
    Args:
        path_to_zip_file (str): 압축파일 경로
        directory_to_extract_to (str): 압축 풀고 저장할 경로
    """
    import zipfile
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)


In [ ]:
from google.colab import drive
drive.mount("/content/drive")



Mounted at /content/drive


In [ ]:
import shutil
shutil.copyfile("/content/drive/MyDrive/고려대학교/기계학습개론/url_encoded_dataset.csv", "/content/url_encoded_dataset.csv")
shutil.copyfile("/content/drive/MyDrive/고려대학교/기계학습개론/tcp_payload_RNN.model.zip", "/content/tcp_payload_RNN.model.zip")
shutil.copyfile("/content/drive/MyDrive/고려대학교/기계학습개론/http_data_RNN.model.zip", "/content/http_data_RNN.model.zip")

'/content/url_encoded_dataset.csv'

In [ ]:
data = pd.read_csv("/content/url_encoded_dataset.csv").sample(frac=1)
data = data.drop(["Unnamed: 0", "http.request.uri.query", "http.file_data", "http.request.full_uri", 
                  "tcp.options", "http.request.uri.path", "http.request.uri.query.dict", "http.file_data.dict", "http.request.uri.scheme"], axis="columns")
data["http.data"] = data["http.data"].fillna("")
original_data = data.copy()

# RNN학습시, URL정보를 바탕으로 분류를 진행할 수 있으므로 URL 정보를 삭제해야함.
data["tcp.payload"] = original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([_ for _ in x.split(" ") if not (_.startswith("/DVWA") and _.endswith(".php"))]))

<ipython-input-5-776e6cc6f1c7>:1: DtypeWarning: Columns (3,6,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/url_encoded_dataset.csv").sample(frac=1)


In [ ]:
# Tokenize values
tokenizer = Tokenizer(oov_token="<NUMBER>")
tokenizer.fit_on_texts(original_data["http.data"])
# word_index = tokenizer.word_index
# word_index = {key: val for key, val in word_index.items() if not re.fullmatch("\d+",key)} # erase number tokenized.
# tokenizer.word_index = {key: i for i, key in enumerate(word_index.keys())} # re index
padded = pad_sequences(tokenizer.texts_to_sequences(original_data["http.data"]), padding="post", value=0)
data["http.data"] = padded.tolist()

In [ ]:
# Tokenize tcp.payload
tokenizer_payload = Tokenizer(oov_token="<PAD>")
tokenizer_payload.fit_on_texts(original_data["tcp.payload"].astype("str"))
padded = pad_sequences(tokenizer_payload.texts_to_sequences(original_data["tcp.payload"].astype("str")), padding="post", value=0)
data["tcp.payload"] = padded.tolist()

In [ ]:
data.head()

,ip.src_host,frame.time,tcp.checksum,http.response,arp.src.proto_ipv4,http.content_length,Attack_type,arp.dst.proto_ipv4,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,http.referer,ip.dst_host,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.payload,tcp.srcport,arp.opcode,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
33778,192.168.0.170,2021 18:42:14.652443000,0x0000eaac,0.0,0,0.0,Password,0,0.0,0.0,86.0,1.0,0.0,0.0,192.168.0.128,1.0,3.897315e+09,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",36678.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
91226,192.168.0.170,2021 19:38:16.158491000,0x00006d5e,0.0,0,0.0,XSS,0,0.0,0.0,0.0,0.0,1.0,0,192.168.0.128,0.0,0.000000e+00,0.0,80.0,0x00000002,0.0,0.0,"[36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",32796.0,0.0,True,False,False,False,False,False,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
26703,192.168.0.170,2021 18:14:57.432012000,0x00008e32,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,3.795402e+09,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",44856.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
83406,192.168.0.128,2021 18:36:44.752969000,0x000068b6,0.0,0,0.0,XSS,0,0.0,0.0,0.0,441.0,0.0,0,192.168.0.170,1.0,3.240978e+09,1.0,59858.0,0x00000011,103.0,0.0,"[36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",80.0,0.0,True,False,False,False,False,False,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
69315,192.168.0.170,2021 20:41:17.194102000,0x0000edad,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,3.665398e+09,0.0,80.0,0x00000018,1.0,0.0,"[18, 10, 3, 15, 6, 5, 2, 11, 7, 8, 2, 9, 12, 1...",52768.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
original_data.head()

,ip.src_host,frame.time,tcp.checksum,http.response,arp.src.proto_ipv4,http.content_length,Attack_type,arp.dst.proto_ipv4,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,http.referer,ip.dst_host,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.payload,tcp.srcport,arp.opcode,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
33778,192.168.0.170,2021 18:42:14.652443000,0x0000eaac,0.0,0,0.0,Password,0,0.0,0.0,86.0,1.0,0.0,0.0,192.168.0.128,1.0,3.897315e+09,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,36678.0,0.0,False,True,False,False,True,False,192.168.0.128,
91226,192.168.0.170,2021 19:38:16.158491000,0x00006d5e,0.0,0,0.0,XSS,0,0.0,0.0,0.0,0.0,1.0,0,192.168.0.128,0.0,0.000000e+00,0.0,80.0,0x00000002,0.0,0.0,NaN,32796.0,0.0,True,False,False,False,False,False,NaN,
26703,192.168.0.170,2021 18:14:57.432012000,0x00008e32,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,3.795402e+09,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,44856.0,0.0,False,True,False,False,True,False,192.168.0.128,
83406,192.168.0.128,2021 18:36:44.752969000,0x000068b6,0.0,0,0.0,XSS,0,0.0,0.0,0.0,441.0,0.0,0,192.168.0.170,1.0,3.240978e+09,1.0,59858.0,0x00000011,103.0,0.0,NaN,80.0,0.0,True,False,False,False,False,False,NaN,
69315,192.168.0.170,2021 20:41:17.194102000,0x0000edad,0.0,0,0.0,Password,0,0.0,0.0,221.0,1.0,0.0,0.0,192.168.0.128,1.0,3.665398e+09,0.0,80.0,0x00000018,1.0,0.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,52768.0,0.0,False,True,False,False,True,False,192.168.0.128,


In [ ]:
def zipFile(fileName):
    import shutil
    shutil.make_archive(fileName,"zip", fileName)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3)

In [ ]:
labels = {key: i for i, key in enumerate(test["Attack_type"].unique())}
def to_onehot(x):
    l = [0 for _ in range(labels.__len__())]
    l[labels[x]] = 1
    return l

In [ ]:
payload_test_X ,RNN_test_X, test_X, test_Y =  np.stack(test["tcp.payload"]),np.stack(test["http.data"]), test.drop(["Attack_type", "http.data"], axis="columns"), np.stack(test["Attack_type"].apply(to_onehot))
payload_train_X ,RNN_train_X, train_X, train_Y =  np.stack(train["tcp.payload"]),np.stack(train["http.data"]), train.drop(["Attack_type", "http.data"], axis="columns"), np.stack(train["Attack_type"].apply(to_onehot))

In [ ]:
vocab_size = tokenizer.word_index.keys().__len__()
input_dim = RNN_train_X.shape[1]

In [ ]:
# layers=[
# keras.layers.Embedding(1000, 128, input_length=1000),
# keras.layers.LSTM(64, activation="tanh", dropout=0.5, return_sequences=True),
# # keras.layers.Dense(100, activation="relu"),
# # keras.layers.Dense(100, activation="relu"),
# keras.layers.Dense(3, activation="softmax")
# ]
import os
RNN_MODEL_PATH:str = "logs/http_data_RNN.model"
if "http_data_RNN.model.zip" in os.listdir(os.getcwd()):
    unzip(f"{RNN_MODEL_PATH}.zip", f"{RNN_MODEL_PATH}")
    RNN_model = keras.models.load_model(f"{RNN_MODEL_PATH}")
else:
    RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(vocab_size, 128, input_length=input_dim),
            keras.layers.LSTM(128, dropout=0.7, recurrent_dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    RNN_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
        optimizer=tf.keras.optimizers.Adam(learning_rate=.005), 
        metrics=["accuracy"],
    )
    RNN_model.fit(RNN_train_X, train_Y, epochs=5, batch_size=128)
    RNN_model.save("http_data_RNN.model")
    zipFile("http_data_RNN.model")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


510/510 [==============================] - 177s 329ms/step - loss: 0.6270 - accuracy: 0.7675
Epoch 2/10
510/510 [==============================] - 149s 292ms/step - loss: 0.5210 - accuracy: 0.8131
Epoch 3/10
510/510 [==============================] - 146s 287ms/step - loss: 0.3704 - accuracy: 0.8406
Epoch 4/10
510/510 [==============================] - 148s 290ms/step - loss: 0.3682 - accuracy: 0.8406
Epoch 5/10
510/510 [==============================] - 147s 289ms/step - loss: 0.3680 - accuracy: 0.8406
Epoch 6/10
510/510 [==============================] - 147s 289ms/step - loss: 0.3678 - accuracy: 0.8406
Epoch 7/10
510/510 [==============================] - 147s 288ms/step - loss: 0.3680 - accuracy: 0.8406
Epoch 8/10
510/510 [==============================] - 147s 289ms/step - loss: 0.3678 - accuracy: 0.8406
Epoch 9/10
510/510 [==============================] - 146s 287ms/step - loss: 0.3679 - accuracy: 0.8406
Epoch 10/10
510/510 [==============================] - 147s 289ms/step - lo

In [ ]:
vocab_size = tokenizer_payload.word_index.keys().__len__()
input_dim = payload_train_X.shape[1]

In [ ]:
PAYLOAD_RNN_MODEL_PATH:str = "logs/tcp_payload_RNN.model"
if "tcp_payload_RNN.model.zip" in os.listdir(os.getcwd()):
    unzip(f"{PAYLOAD_RNN_MODEL_PATH}.zip", f"{PAYLOAD_RNN_MODEL_PATH}")
    payload_RNN_model = keras.models.load_model(f"{PAYLOAD_RNN_MODEL_PATH}")
else:
    payload_RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(vocab_size, 128, input_length=input_dim),
            keras.layers.LSTM(128, dropout=0.7, recurrent_dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    payload_RNN_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
        optimizer="adam", 
        metrics=["accuracy"],
    )
    payload_RNN_model.fit(payload_train_X, train_Y, epochs=5, batch_size=32)
    payload_RNN_model.save("tcp_payload_RNN.model")
    zipFile("tcp_payload_RNN.model")

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 665/2040 [========>.....................] - ETA: 9:56 - loss: 0.7321 - accuracy: 0.7635

In [ ]:
file_data_prediction = RNN_model.predict(RNN_test_X)

In [ ]:
payload_prediction = payload_RNN_model.predict(payload_test_X)

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

In [ ]:
# http.file_data Prediction
precision_score(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1), average="macro")

In [ ]:
recall_score(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1), average="macro")

In [ ]:
confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1))

In [ ]:
# tcp.payload Prediction
precision_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")

In [ ]:
recall_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")

In [ ]:
confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1))

In [ ]:
shutil.copyfile("/content/tcp_payload_RNN.model.zip", "/content/drive/MyDrive/고려대학교/기계학습개론/tcp_payload_RNN.model.zip")
shutil.copyfile("/content/http_data_RNN.model.zip", "/content/drive/MyDrive/고려대학교/기계학습개론/http_data_RNN.model.zip")